# 06. Decoding Strategies

Давайте посмотрим, как можно манипулировать параметрами декодинга генеративной модели, чтобы получить желаемое поведение. Такие методы называются стратегиями декодирования. [Исчерпывающий туториал](https://huggingface.co/blog/how-to-generate) по этой теме, из которого я взял некоторые части.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model_for_decode = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(torch_device)

# encode context the generation is conditioned on
model_inputs = tokenizer('I enjoy walking with my cute dog', return_tensors='pt').to(torch_device)

### 1. Greedy search

Самый простой в реализации и запуске: на каждом шаге брать наиболее вероятный токен:

![](https://huggingface.co/blog/assets/02_how-to-generate/greedy_search.png)

In [ ]:
# generate 40 new tokens
greedy_output = model_for_decode.generate(**model_inputs, max_new_tokens=40)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure


### 2. Beam search

Beam search поддерживает несколько гипотез одновременно, выбирая N лучших кандидатов на каждом этапе (num_beam=2 на рис):

![](https://huggingface.co/blog/assets/02_how-to-generate/beam_search.png)

In [ ]:
# activate beam search and early_stopping
beam_output = model_for_decode.generate(
    **model_inputs,
    max_new_tokens=40,
    num_beams=5,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I'm not sure if I'll ever be able to walk with him again. I'm not sure


Одной из важных особенностей beam search является то, что мы можем сравнивать лучшие пути  после генерации и выбирать тот путь, который лучше всего подходит для нашей цели.

В трансформаторах мы просто устанавливаем параметр num_return_sequences на количество путей с наивысшей вероятностью. Однако убедитесь, что num_return_sequences <= num_beams!

In [ ]:
# set return_num_sequences > 1
beam_outputs = model_for_decode.generate(
    **model_inputs,
    max_new_tokens=40,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to
1: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to
2: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's a good idea to
3: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time to take a
4: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's a good idea

### 3.1 Сэмплинг

Декодинг через сэмплинг учитывает вероятность всех слов, и возвращает их согласно вероятности. То есть, даже если слово не самое вероятное, оно может быть случайно выбранным. В отличие от предыдущих стратегий, эта уже не является детерминированной

![](https://huggingface.co/blog/assets/02_how-to-generate/sampling_search.png)

In [ ]:
# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model_for_decode.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=0,
    temperature=0.6
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I think I'm a bit too shy for my pet. I'm not totally sure what to do about it, but I feel like I can handle it.

My favorite part of


### 3.1 Top-K

При сэмплинге через Top-K фильтруются K наиболее вероятных следующих слов, и функция вероятностей  перераспределяется только среди этих K следующих слов. GPT2 принял эту схему выборки, что стало одной из причин ее успеха в генерации историй.

![](https://huggingface.co/blog/assets/02_how-to-generate/top_k_sampling.png)

In [ ]:
# set top_k to 50
sample_output = model_for_decode.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

### Top-p (nucleus) sampling

Вместо выборки только из наиболее вероятных слов K, Top-p  выбирает из наименьшего возможного набора слов, кумулятивная вероятность которых превышает вероятность p. Затем  вероятности перераспределяются среди этого набора слов. Таким образом, размер набора слов (т.е. количество слов в наборе) может динамически увеличиваться и уменьшаться в соответствии с распределением вероятности следующего слова:

![](https://huggingface.co/blog/assets/02_how-to-generate/top_p_sampling.png)

In [ ]:
sample_output = model_for_decode.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_p=0.92,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))
